In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import plotly.graph_objects as go

import random
import pandas as pd
import numpy as np

!pip install torchinfo
from torchinfo import summary

In [23]:
# Random Seed 고정 (학습 반복 시행 시에도 동일한 결과가 나오도록)

seed = 20250308

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

**1. 데이터셋 로딩 및 데이터 분석**

In [24]:
# 데이터셋 로딩

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = torchvision.datasets.MNIST(root='./data',
                                           train=True,
                                           transform=transform,
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data',
                                          train=False,
                                          transform=transform,
                                          download=True)


In [25]:
# 시간 절약을 위해, 학습 데이터에서 랜덤하게 일부 샘플만 추출

from torch.utils.data import Subset, DataLoader

NUM_TRAIN_SAMPLES = 8000
BATCH_SIZE = 32

subset_indices = random.sample(range(len(train_dataset)), NUM_TRAIN_SAMPLES)
train_subset = Subset(train_dataset, subset_indices)

train_loader = DataLoader(train_subset,
                          batch_size=BATCH_SIZE,
                          shuffle=True)

# 테스트 데이터셋은 학습 대상이 아니므로 그대로 이용
test_loader = DataLoader(test_dataset,
                         batch_size=BATCH_SIZE,
                         shuffle=False)

In [26]:
# 클래스 불균형 분석

# 학습 데이터
train_labels = torch.tensor([train_subset.dataset.targets[i] for i in subset_indices])
train_class_counts = torch.bincount(train_labels)
print(train_class_counts)

NUM_CLASSES = len(train_class_counts)

tensor([777, 920, 795, 805, 782, 753, 835, 795, 744, 794])


In [27]:
train_class_percentage = np.array(train_class_counts) * 100.0 / sum(train_class_counts)

train_y_distrib = pd.DataFrame({'class': list(range(NUM_CLASSES)),
                                'count': train_class_counts,
                                'percentage (%)': train_class_percentage})

train_y_distrib

,class,count,percentage (%)
0,0,777,9.712500
1,1,920,11.500001
2,2,795,9.937500
3,3,805,10.062500
4,4,782,9.775000
5,5,753,9.412500
6,6,835,10.437500
7,7,795,9.937500
8,8,744,9.300000
9,9,794,9.925000


In [28]:
# 테스트 데이터
test_labels = test_loader.dataset.targets
test_class_counts = torch.bincount(test_labels)
print(test_class_counts)

tensor([ 980, 1135, 1032, 1010,  982,  892,  958, 1028,  974, 1009])


In [29]:
test_class_percentage = np.array(test_class_counts) * 100.0 / sum(test_class_counts)

test_y_distrib = pd.DataFrame({'class': list(range(NUM_CLASSES)),
                               'count': test_class_counts,
                               'percentage (%)': test_class_percentage})

test_y_distrib

,class,count,percentage (%)
0,0,980,9.80
1,1,1135,11.35
2,2,1032,10.32
3,3,1010,10.10
4,4,982,9.82
5,5,892,8.92
6,6,958,9.58
7,7,1028,10.28
8,8,974,9.74
9,9,1009,10.09


In [30]:
# CNN 모델 정의

class CNN(nn.Module):

    def __init__(self, padding_options):
        super(CNN, self).__init__()
        self.padding_options = padding_options

        padding_conv1 = padding_options[0]
        padding_conv2 = padding_options[1]
        padding_conv4 = padding_options[2]

        # Conv
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, padding=2, padding_mode=padding_conv1),
            nn.ReLU()
        )
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, padding=2, padding_mode=padding_conv2),
            nn.ReLU()
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3),
            nn.ReLU()
        )
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv4 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=5, padding=2, padding_mode=padding_conv4),
            nn.ReLU()
        )
        self.conv5 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3),
            nn.ReLU()
        )

        # Fully Connected
        self.fc1 = nn.Sequential(
            nn.Linear(256 * 4 * 4, 256),
            nn.Sigmoid()
        )
        self.fc_final = nn.Sequential(
            nn.Linear(256, 10),
            nn.Softmax()  # Classification Task 의 Output Layer 이므로 Softmax 고정
        )

    def forward(self, x):

        # Conv
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.pool2(x)
        x = self.conv4(x)
        x = self.conv5(x)

        x = x.view(-1, 256 * 4 * 4)

        # Fully Connected
        x = self.fc1(x)
        x = self.fc_final(x)

        return x

In [31]:
# 모델 구조 출력

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN(padding_options=['zeros', 'reflect', 'replicate']).to(device)

print(summary(model, input_size=(BATCH_SIZE, 1, 28, 28)))

Layer (type:depth-idx)                   Output Shape              Param #
CNN                                      [32, 10]                  --
├─Sequential: 1-1                        [32, 32, 28, 28]          --
│    └─Conv2d: 2-1                       [32, 32, 28, 28]          832
│    └─ReLU: 2-2                         [32, 32, 28, 28]          --
├─MaxPool2d: 1-2                         [32, 32, 14, 14]          --
├─Sequential: 1-3                        [32, 64, 14, 14]          --
│    └─Conv2d: 2-3                       [32, 64, 14, 14]          51,264
│    └─ReLU: 2-4                         [32, 64, 14, 14]          --
├─Sequential: 1-4                        [32, 64, 12, 12]          --
│    └─Conv2d: 2-5                       [32, 64, 12, 12]          36,928
│    └─ReLU: 2-6                         [32, 64, 12, 12]          --
├─MaxPool2d: 1-5                         [32, 64, 6, 6]            --
├─Sequential: 1-6                        [32, 128, 6, 6]           --
│    └

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1844: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return inner()


**3. 데이터셋 분리**

* Train Data -> Train Data + Valid Data (epoch) + Valid Data (하이퍼파라미터 최적화)

In [32]:
# 데이터셋 분리

from torch.utils.data import random_split

# 샘플 수
num_train = 1000
num_valid_epoch = 2000
num_valid_hpo = 5000

assert NUM_TRAIN_SAMPLES == num_train + num_valid_epoch + num_valid_hpo

# 데이터셋 분리
train_dataset, valid_epoch_dataset, valid_hpo_dataset =\
    random_split(train_subset, [num_train, num_valid_epoch, num_valid_hpo])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_epoch_loader = DataLoader(valid_epoch_dataset, batch_size=BATCH_SIZE, shuffle=False)
valid_hpo_loader = DataLoader(valid_hpo_dataset, batch_size=BATCH_SIZE, shuffle=False)

**4. 하이퍼파라미터 최적화 학습 실시 함수**

* 하이퍼파라미터 최적화 라이브러리는 Optuna 사용
* 하이퍼파라미터 탐색 200 회 실시
* 하이퍼파라미터 목록
  * 레이어 별 Padding 의 종류
    * 레이어 : Conv1, Conv2, Conv4
    * Padding 의 종류 : zeros, reflect, replicate, circular
  * Learning Rate
    * 0.0001 ~ 0.003 (= 1e-4 ~ 3e-3) 범위


In [33]:
MAX_EPOCHS = 65536
TRIAL_COUNT = 200           # HPO trial count
EARLY_STOPPING_ROUNDS = 10  # Early Stopping epoch count

In [34]:
from sklearn.metrics import accuracy_score
from copy import deepcopy

In [35]:
# Optuna 설정

!pip install optuna
import optuna
import logging

optuna.logging.set_verbosity(logging.WARNING)

In [36]:
# 모델 학습 실시

# args :
# - model           : 학습할 모델
# - train_loader    : Training Data Loader
# - train_loss_list : 각 epoch 에서의 train loss 기록

# returns :
# - train_loss : 모델의 Train Loss

def run_train(model, train_loader, train_loss_list):
    model.train()
    train_loss = 0.0
    cnt = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # train 실시
        model.optimizer.zero_grad()
        outputs = model(images)

        loss = nn.CrossEntropyLoss()(outputs, labels)
        loss.backward()
        model.optimizer.step()

        train_loss += loss.item()
        cnt += 1

    train_loss_list.append(train_loss / len(train_loader))
    return train_loss_list[-1]

In [37]:
# 모델 validation 실시

# args :
# - model        : validation 할 모델
# - valid_loader : Validation Data Loader
# - during_train : 모델 학습 중이면 True, 그렇지 않으면 False

# returns :
# - val_accuracy : 모델의 validation 정확도
# - val_loss     : 모델의 validation loss

def run_validation(model, valid_loader, during_train=True):
    model.eval()
    correct, total = 0, 0
    val_loss_sum = 0

    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            val_loss_batch = nn.CrossEntropyLoss(reduction='sum')(outputs, labels)
            val_loss_sum += val_loss_batch

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        # Accuracy 계산
        val_accuracy = correct / total
        val_loss = val_loss_sum / total

    return val_accuracy, val_loss

In [38]:
# 모델 학습 및 validation 전체 프로세스

# args :
# - model              : 학습할 모델
# - train_loader       : Training Data Loader
# - valid_epoch_loader : 각 epoch 마다 validation 할 Valid Data Loader
# - valid_hpo_loader   : 최종적으로 해당 하이퍼파라미터 조합에 대한 Valid Data Loader
# - verbose            : 학습 중 프로세스 출력 여부

# returns :
# - final_acc        : 해당 하이퍼파라미터 조합에 대한 최종 Accuracy (valid accuracy 가 가장 높았던 epoch 의 모델로 측정)
# - best_epoch_model : valid accuracy 가 가장 높았던 epoch 에서 생성된 모델
# - epochs           : 해당 학습의 총 epoch count

def run_model_common(model, train_loader, valid_epoch_loader, valid_hpo_loader,
                     verbose=False):

    train_loss_list = []        # train loss
    valid_acc_list = []         # valid accuracy
    valid_loss_list = []        # valid loss

    max_valid_acc = 0.0         # max validation accuracy
    min_valid_loss = None       # min validation loss

    best_valid_acc_epoch = -1   # valid accuracy 가 가장 높았던 epoch
    best_valid_loss_epoch = -1  # valid loss 가 가장 낮았던 epoch
    best_epoch_model = None     # valid accuracy 가 가장 높았던 epoch 의 모델

    # 1. 학습 실시
    for epoch in range(MAX_EPOCHS):

        # 1-1. train model
        train_loss = run_train(model, train_loader, train_loss_list)

        # 1-2. validate model (with EPOCH VALID SET)
        epoch_acc, val_loss = run_validation(model, valid_epoch_loader)
        valid_acc_list.append(epoch_acc)
        valid_loss_list.append(val_loss)

        # 1-3. Early Stopping 처리 (overfitting 방지)
        if epoch_acc > max_valid_acc:
            max_valid_acc = epoch_acc
            best_valid_acc_epoch = epoch

            best_epoch_model = CNN(padding_options=model.padding_options).to(device)
            best_epoch_model.load_state_dict(model.state_dict())

            if verbose:
                print('best model updated')

        # Early Stopping 학습 종료
        if epoch - best_valid_acc_epoch >= EARLY_STOPPING_ROUNDS:
            epochs = epoch
            break

        # 1-4. 결과 출력
        if verbose:
            print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Valid Loss: {val_loss:.4f}, Valid Accuracy: {epoch_acc:.4f}")

    # check best-epoch model correctly loaded
    checked_acc, _ = run_validation(best_epoch_model,
                                    valid_epoch_loader,
                                    during_train=False)

    if verbose:
        print(f"Best Epoch: {best_valid_acc_epoch}, Best Valid Acc: {max_valid_acc}")
        print(f"Valid Acc (with Epoch valid set) on Loaded Best Model: {checked_acc}")

    assert abs(max_valid_acc - checked_acc) < 1e-8

    # 2. validate best-epoch model (with HPO VALID SET)
    final_acc, _ = run_validation(best_epoch_model,
                                  valid_hpo_loader,
                                  during_train=False)

    if verbose:
        print(f"Final Acc (with HPO valid set) on Loaded Best Model: {final_acc}")

    return final_acc, best_epoch_model, epochs

In [39]:
print(device)

cuda


**4-1. 실험 실시**

In [40]:
hpo_best_acc = 0              # 모든 Hyper-param 조합의 HPO Valid set 정확도 중 가장 높은 것
best_hyperparam_set = None    # HPO Valid set 정확도가 가장 높은 Hyper-param 조합
best_hyperparam_model = None  # best_hyperparam_set 의 Hyper-param 조합으로 학습된 모델

In [41]:
trial_count = 0   # 1st ~ 10th trial 에만 학습 중 정보 출력
epoch_count = []  # 각 trial 의 epoch 횟수 리스트

def objective(trial):
    global hpo_best_acc, best_hyperparam_set, best_hyperparam_model, trial_count, epoch_count

    # hyper-params
    padding_modes = ['zeros', 'reflect', 'replicate', 'circular']

    params = {
        'padding_mode_conv1': trial.suggest_categorical('padding_mode_conv1', padding_modes),
        'padding_mode_conv2': trial.suggest_categorical('padding_mode_conv2', padding_modes),
        'padding_mode_conv4': trial.suggest_categorical('padding_mode_conv4', padding_modes),
        'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.003, log=True)
    }

    # define and run model
    pad_conv1 = params['padding_mode_conv1']
    pad_conv2 = params['padding_mode_conv2']
    pad_conv4 = params['padding_mode_conv4']

    model = CNN(padding_options=[pad_conv1, pad_conv2, pad_conv4]).to(device)

    model.optimizer = torch.optim.AdamW(model.parameters(),
                                        lr=params['learning_rate'])

    final_acc, best_epoch_model, epochs = run_model_common(model,
                                                           train_loader,
                                                           valid_epoch_loader,
                                                           valid_hpo_loader,
                                                           verbose=(trial_count < 10))

    trial_count += 1
    epoch_count.append(epochs)

    # global best model 갱신
    if final_acc > hpo_best_acc:
        hpo_best_acc = final_acc
        best_hyperparam_set = params

        best_hyperparam_model = CNN(padding_options=[pad_conv1, pad_conv2, pad_conv4]).to(device)
        best_hyperparam_model.load_state_dict(best_epoch_model.state_dict())

        print(f'best_hyperparam_model updated with Accuracy={hpo_best_acc:.4f}')

    print(f"Trial: {trial_count}, Params: {params}, Accuracy: {final_acc:.4f}")
    return final_acc

In [42]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=TRIAL_COUNT)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


best model updated
Epoch 1, Train Loss: 2.2286, Valid Loss: 2.0138, Valid Accuracy: 0.5475
best model updated
Epoch 2, Train Loss: 1.9195, Valid Loss: 1.8875, Valid Accuracy: 0.6330
best model updated
Epoch 3, Train Loss: 1.8002, Valid Loss: 1.7789, Valid Accuracy: 0.7575
best model updated
Epoch 4, Train Loss: 1.7102, Valid Loss: 1.7284, Valid Accuracy: 0.7865
best model updated
Epoch 5, Train Loss: 1.6718, Valid Loss: 1.6808, Valid Accuracy: 0.8155
best model updated
Epoch 6, Train Loss: 1.6333, Valid Loss: 1.6561, Valid Accuracy: 0.8285
best model updated
Epoch 7, Train Loss: 1.6096, Valid Loss: 1.6424, Valid Accuracy: 0.8410
best model updated
Epoch 8, Train Loss: 1.6034, Valid Loss: 1.6359, Valid Accuracy: 0.8475
best model updated
Epoch 9, Train Loss: 1.5885, Valid Loss: 1.6242, Valid Accuracy: 0.8505
Epoch 10, Train Loss: 1.5795, Valid Loss: 1.6263, Valid Accuracy: 0.8470
Epoch 11, Train Loss: 1.5829, Valid Loss: 1.6337, Valid Accuracy: 0.8350
best model updated
Epoch 12, Train 

In [73]:
# Test Dataset 성능 평가

print(f'best hyper-param: {best_hyperparam_set}, best acc: {hpo_best_acc}')

best hyper-param: {'padding_mode_conv1': 'replicate', 'padding_mode_conv2': 'zeros', 'padding_mode_conv4': 'replicate', 'learning_rate': 0.0012288870734115834}, best acc: 0.964


In [74]:
# best_hyperparam_model 이 정상적으로 load 되었는지 최종 확인

checked_hpo_acc, _ = run_validation(best_hyperparam_model,
                                    valid_hpo_loader,
                                    during_train=False)

print(f"Valid Acc (with HPO valid set) on Best Hyper-param Model: {checked_hpo_acc}")

assert abs(hpo_best_acc - checked_hpo_acc) < 1e-8

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1736: UserWarning:

Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.



Valid Acc (with HPO valid set) on Best Hyper-param Model: 0.964


In [75]:
# 테스트셋에 대한 최종 정확도

hpo_final_acc, _ = run_validation(best_hyperparam_model,
                                  test_loader,
                                  during_train=False)

print(f'Final HPO Acc (with test set) : {hpo_final_acc}')

Final HPO Acc (with test set) : 0.964


**5. HPO 성능 결과 확인**

In [76]:
from optuna.visualization import plot_optimization_history

In [110]:
# HPO 추이

fig = plot_optimization_history(study)
fig.update_layout(width=1000,
                  height=650,
                  yaxis_title='Accuracy (HPO valid set)')
fig.show()

In [111]:
fig.update_layout(yaxis=dict(range=[0.925, 0.965]))
fig.show()

**6. 각 Hyperparameter 값에 따른 성능 분포 확인**

In [112]:
# trial DataFrame 가져오기

trials_df = study.trials_dataframe()

In [113]:
trials_df

,number,value,datetime_start,datetime_complete,duration,params_learning_rate,params_padding_mode_conv1,params_padding_mode_conv2,params_padding_mode_conv4,state
0,0,0.9536,2025-03-08 08:25:00.062600,2025-03-08 08:25:58.176295,0 days 00:00:58.113695,0.000418,circular,reflect,zeros,COMPLETE
1,1,0.9464,2025-03-08 08:25:58.176457,2025-03-08 08:26:42.144727,0 days 00:00:43.968270,0.001086,reflect,circular,circular,COMPLETE
2,2,0.9488,2025-03-08 08:26:42.144900,2025-03-08 08:27:20.261806,0 days 00:00:38.116906,0.000364,reflect,reflect,replicate,COMPLETE
3,3,0.9508,2025-03-08 08:27:20.262000,2025-03-08 08:27:58.480260,0 days 00:00:38.218260,0.000612,zeros,circular,zeros,COMPLETE
4,4,0.9528,2025-03-08 08:27:58.480408,2025-03-08 08:28:28.923771,0 days 00:00:30.443363,0.000596,zeros,reflect,replicate,COMPLETE
...,...,...,...,...,...,...,...,...,...,...
195,195,0.9450,2025-03-08 10:26:28.344950,2025-03-08 10:27:15.908422,0 days 00:00:47.563472,0.000857,replicate,reflect,replicate,COMPLETE
196,196,0.9580,2025-03-08 10:27:15.908610,2025-03-08 10:27:52.987356,0 days 00:00:37.078746,0.000747,replicate,zeros,replicate,COMPLETE
197,197,0.9454,2025-03-08 10:27:52.987561,2025-03-08 10:28:46.260336,0 days 00:00:53.272775,0.001077,circular,replicate,replicate,COMPLETE
198,198,0.9548,2025-03-08 10:28:46.260525,2025-03-08 10:29:29.813648,0 days 00:00:43.553123,0.000959,replicate,zeros,circular,COMPLETE


In [114]:
# 차트 표시 순서를 일정하게

category_orders = ['zeros', 'reflect', 'replicate', 'circular']

In [115]:
# Early Stopping Type 별, Learning Rate 에 따른 Accuracy 분포

import plotly.express as px

fig = px.scatter(trials_df,
                 x="params_learning_rate",
                 y="value",
                 color="params_padding_mode_conv1",
                 category_orders={'params_padding_mode_conv1': category_orders},
                 title="Accuracy Distribution by Conv.1 Padding Type & Learning Rate")

fig.update_layout(width=1000, height=600,
                  xaxis_title='Learning Rate',
                  yaxis_title='Accuracy')

fig.show()

In [116]:
fig.update_layout(yaxis=dict(range=[0.925, 0.965]))
fig.show()

In [117]:
fig = px.scatter(trials_df,
                 x="params_learning_rate",
                 y="value",
                 color="params_padding_mode_conv2",
                 category_orders={'params_padding_mode_conv2': category_orders},
                 title="Accuracy Distribution by Conv.2 Padding Type & Learning Rate")

fig.update_layout(width=1000, height=600,
                  xaxis_title='Learning Rate',
                  yaxis_title='Accuracy')

fig.show()

In [118]:
fig.update_layout(yaxis=dict(range=[0.925, 0.965]))
fig.show()

In [119]:
fig = px.scatter(trials_df,
                 x="params_learning_rate",
                 y="value",
                 color="params_padding_mode_conv4",
                 category_orders={'params_padding_mode_conv4': category_orders},
                 title="Accuracy Distribution by Conv.4 Padding Type & Learning Rate")

fig.update_layout(width=1000, height=600,
                  xaxis_title='Learning Rate',
                  yaxis_title='Accuracy')

fig.show()

In [120]:
fig.update_layout(yaxis=dict(range=[0.925, 0.965]))
fig.show()